In [1]:
from tools.data import DreemDatasets
from preprocessing.features import ExtractFeatures
import numpy as np
import matplotlib.pyplot as plt
from models.cnn import CNN
import torch
import torch.utils.data
import torch.optim as optim
from tools.trainer import CNNTrainer as Trainer

In [2]:
use_datasets = ['eeg_1', 'eeg_2', 'eeg_3', 'eeg_4', 'eeg_5', 'eeg_6', 'eeg_7']
batch_size = 32

In [3]:
train_set, val_set = DreemDatasets('dataset/train.h5', 'dataset/train_y.csv', 
                                   keep_datasets=use_datasets, split_train_val=0.8, seed=0,
                                   size=5000).get()

train_set.load_data("dataset/eegs_bands/train")  # Charge en mémoire. Peut-être un peu long
val_set.load_data("dataset/eegs_bands/val")  # Charge en mémoire. Peut-être un peu long

train_set.close()  # On ferme les fichiers h5
val_set.close()

Loading data in memory...
5412 in 7 datasets to load
Loading dataset eeg_1 ...
Loading dataset eeg_2 ...
Loading dataset eeg_3 ...
Loading dataset eeg_4 ...
Loading dataset eeg_5 ...
Loading dataset eeg_6 ...
Loading dataset eeg_7 ...
Done.
Loading data in memory...
1353 in 7 datasets to load
Loading dataset eeg_1 ...
Loading dataset eeg_2 ...
Loading dataset eeg_3 ...
Loading dataset eeg_4 ...
Loading dataset eeg_5 ...
Loading dataset eeg_6 ...
Loading dataset eeg_7 ...
Done.


In [4]:
X, _, y = train_set[:]
X_val, _, y_val = train_set[:]

X = X.transpose(1, 0, 2, 3)
X_val = X_val.transpose(1, 0, 2, 3)

X = X.reshape(-1, 7*4, 1500)
X_val = X_val.reshape(-1, 7*4, 1500)

y = y.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

print(y.shape)

(5412, 1)


In [5]:
train_loader = torch.utils.data.DataLoader(train_set.torch_dataset(), 
                                           batch_size=batch_size, 
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set.torch_dataset(), batch_size=batch_size)

In [6]:
number_groups = 3
channels = [128, 64, 32]
kernel_sizes = [100, 50, 50]
kernel_pooling = [5, 2, 2]
use_cuda = True

def transform(data, _):
    data = data.view(-1, 7*4, 1500)
    return data, None


In [7]:
model = CNN(in_features=1500, out_features=5, in_channels=7*4,
            number_groups=number_groups, size_groups=1, hidden_channels=channels, kernel_sizes=kernel_sizes,
            kernel_pooling=kernel_pooling)
if use_cuda:
    model.cuda()

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

trainer = Trainer(train_loader, val_loader, optimizer, model_50hz=model,
                  log_every=10, save_folder='builds', transform=transform)

trainer.train(n_epochs=50)

  0%|          | 0/169.125 [00:00<?, ?it/s]/usr/lib/python3.7/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Train - Epoch 1: : 170it [00:21,  7.76it/s, Loss: 1.6202894449234009]                           
Val - Epoch 1: : 43it [00:03, 14.10it/s, Loss: 1.6980857849121094]                            



Validation set: Average loss: 1.6669, Accuracy: 343/1353 (25%)



Train - Epoch 2:   1%|          | 1/169.125 [00:00<00:27,  6.08it/s, Loss: 1.6395312547683716]


Saved models in builds/2018-12-03 13:52:17.


Train - Epoch 2: : 170it [00:19,  8.60it/s, Loss: 1.6123155355453491]                           
Val - Epoch 2: : 43it [00:02, 14.98it/s, Loss: 1.663747787475586]                             
Train - Epoch 3:   0%|          | 0/169.125 [00:00<?, ?it/s]

...


Validation set: Average loss: 1.6257, Accuracy: 401/1353 (29%)


Saved models in builds/2018-12-03 13:52:17.


Train - Epoch 17: : 170it [00:20,  8.31it/s, Loss: 1.3247218132019043]                           
Val - Epoch 17: : 43it [00:04,  9.14it/s, Loss: 1.6989020109176636]                            
Train - Epoch 18:   0%|          | 0/169.125 [00:00<?, ?it/s]


Validation set: Average loss: 1.6337, Accuracy: 389/1353 (28%)


Saved models in builds/2018-12-03 13:52:17.


Train - Epoch 18:  28%|██▊       | 48/169.125 [00:07<00:23,  5.14it/s, Loss: 1.620147466659546] 


KeyboardInterrupt: 